# Workbench API QA Example Sheet

This Jupyter notebook shows how to programatically connect to the QA data set using our APIs.

QA's data is indexed inside the same lake as our PCAP - Tick History Parquet data, and this notebook will also demonstrate that.

In [ ]:
# You must choose a particular feed and date to use for the queries in this notebook. 
# If you need a list of feeds, go to Help Menu -> "View Feed Data Permissions"

# An example is:
feed = 'bats_edga_a'
dt = '2023-06-07'



In [ ]:
#
# This example shows selecting information the QA source.
#
 
from maystreet_data.api_client import ApiClient

query = """
SELECT
    *
FROM
    treactchg
WHERE
    activationdate >= TIMESTAMP '2022-01-01 00:00:00'
"""

with ApiClient() as client:
    df = client.qa.exec(sql=query, return_type="df")

df.head()

In [ ]:
#
# This example shows selecting information from both our PCAP - Tick History source and our QA source.
#

from maystreet_data.api_client import ApiClient

query = """
SELECT
    avg_aapl_price.dt,
    avg_aapl_price.price,
    treactchg.estpermid,
    treactchg.actgrowth
FROM 
    treactchg
        JOIN
    (
        SELECT 
            dt,
            AVG(price) AS price 
        FROM
            mt_trade 
        WHERE 
            f = '{feed}' 
            AND product = 'AAPL' 
            AND dt = '{dt}'
        GROUP BY
            dt
    ) AS avg_aapl_price ON CAST(CAST(activationdate AS DATE) AS VARCHAR) = avg_aapl_price.dt 
"""

with ApiClient() as client:
    df = client.foundation.exec(sql=query, return_type="df")

df.head()